In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import FinanceDataReader as fdr
import yfinance as yf
import numpy as np

import datetime as dt

from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
import scipy.optimize as sco

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

In [4]:
# 시각화 관련
import plotly_express as px
import plotly.figure_factory as ff
# px를 쉽고 빠르게 붙여주는 모듈 cufflinks
import cufflinks as cf
cf.go_offline(connected=True)
## cf.getThemes()
cf.set_config_file(theme='polar')

import plotly.graph_objects as go

import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
## FRED 정보 수집
from fredapi import Fred

In [6]:
%run ../library/QUANT_FUNCTIONS.ipynb
Q = QUANT()
D = DATA()

## Yield Gap Clock

In [7]:
## 함수에 포함
## fred = Fred(api_key='cfb4f49f5c1a9396f671b8049d992e56')
## GS10 = fred.get_series('DGS10')
## GS2 = fred.get_series('DGS2')
## GS3m = fred.get_series('DTB3')

In [8]:
fred = Fred(api_key='cfb4f49f5c1a9396f671b8049d992e56')
STT = fred.get_series('DTB3')
LTT = fred.get_series('DGS10')
tt = pd.DataFrame([STT, LTT], index=['DTB3', 'DGS10']).T

In [9]:
def get_gab_df(lt = 'DGS10', st = 'DTB3'):
    '''FRED 국채 장단기물 ticker입력시 금리차 장단기추세 데이터로 변환하는 함수'''
    ## STT == DTB3, DGS2 / LTT == DGS10
    fred = Fred(api_key='cfb4f49f5c1a9396f671b8049d992e56')
    STT = fred.get_series(st)
    LTT = fred.get_series(lt)
    
    df = pd.DataFrame([STT, LTT], index=[st, lt]).T
    df = df.fillna(method='pad')
    df = df.dropna()
    
    df['GAP'] = df[lt] - df[st]
    
    ## 추세 대비(10년 이동평균)
    trends = df - (df.rolling(2515).mean())
    trends.columns = df.columns + '_trd'
    ## 전월 대비
    mom = df - (df.shift(20))
    mom.columns = df.columns + '_mom'
    
    new_df = pd.concat([df, trends, mom], axis=1)
    new_df = new_df.dropna()
    ## 월단위 변환
    new_df = D.modi_ts(new_df, 'm')
    new_df = new_df[['GAP', 'GAP_mom', 'GAP_trd']]
    
    ## 시각정보 표현
    new_df['hue'] = 0.1
    new_df['hue'][new_df.index >= new_df.index[-3]] = 0.4
    new_df['hue'][new_df.index == new_df.index[-1]] = 0.5
    new_df['hue'][(new_df['GAP']<=0)&(new_df['GAP_mom']<0)] = 0.2
    new_df['hover_data'] = new_df.index.date.astype(str)
    
    return new_df

In [10]:
df_2y10y = get_gab_df(st='DGS2')

In [11]:
df_2y10y

,GAP,GAP_mom,GAP_trd,hue,hover_data
1986-01-31,1.09,0.07,0.734465,0.1,1986-01-31
1986-02-28,0.42,-0.67,0.065328,0.1,1986-02-28
1986-03-31,0.47,0.03,0.119388,0.1,1986-03-31
1986-04-30,0.55,0.12,0.204155,0.1,1986-04-30
1986-05-30,0.69,0.06,0.349245,0.1,1986-05-30
...,...,...,...,...,...
2021-09-30,1.24,0.15,0.116783,0.1,2021-09-30
2021-10-29,1.07,-0.14,-0.049101,0.1,2021-10-29
2021-11-30,0.91,-0.19,-0.201928,0.4,2021-11-30
2021-12-31,0.79,0.04,-0.313618,0.4,2021-12-31


In [12]:
fig = px.scatter(df_2y10y, x="GAP_mom", y="GAP_trd", color="hue", size='hue', hover_data=['hover_data'])
fig.show()

In [13]:
df_2y10y['GAP'].iplot()

In [14]:
df_2y10y['GAP'].rolling(12).mean().iplot()

## 경기 시계

In [28]:
path = 'E:/경제데이터/'

In [29]:
date = '20210204'

In [ ]:
df_mth = pd.read_pickle(path+'거시지표/월단위/L0_월단위_거시지표_종합_'+date+'.pkl')
df_day = pd.read_pickle(path+'거시지표/일단위/L0_일단위_거시지표_종합_'+date+'.pkl')

In [48]:
## 일단위 데이터 월단위 변환
df_mth = df_mth.interpolate(method='linear', limit=2)
df_dma = df_day.interpolate(method='linear', limit=2).rolling(20).mean() ## 20일 MA로 변환

## 월단위 변환 함수
def day_to_mth(df):
    df = df.copy()
    df['year'] = df.index.year
    df['month'] = df.index.month

    df = df.drop_duplicates(['year', 'month'], keep='last')
    df = df.drop(columns=['year', 'month'])
    return df

## 기준년월 인덱스 처리
tmp_dma = day_to_mth(df_dma)
tmp_dma['기준년월'] = tmp_dma.index.astype(str)
tmp_dma['기준년월'] = tmp_dma['기준년월'].apply(lambda x : x.replace('-', '')[:-2])

In [197]:
df_idc = pd.concat([df_mth.set_index('기준년월'), tmp_dma.set_index('기준년월')], axis = 1)
df_idc.index = pd.to_datetime(df_idc.index, format='%Y%m')
## CPI와 CES는 장기적으로 우상향하므로 증감률 데이터로 변환
df_idc['CES'] = np.log(df_idc['CES']/df_idc['CES'].shift(1))
df_idc['CPI'] = np.log(df_idc['CPI']/df_idc['CPI'].shift(1))

In [227]:
df_idc = df_idc.dropna()

In [229]:
def get_clock_df(df, t = 120):  
    ## 추세 대비(3년 이동평균)
    trends = df - df.rolling(t).mean()
    trends.columns = df.columns + '_trd'
    ## 최근월 대비(1개월 이동평균)
    mom = df - df.shift(1)
    mom.columns = df.columns + '_mom'
    
    new_df = pd.concat([df, trends, mom], axis=1)
    new_df = new_df.dropna()
    
    return new_df

In [230]:
## 증감률 기준으로 산출해야하는 함수
## CES, CPI

In [231]:
clock_df = get_clock_df(df_idc)

In [240]:
## 데이터 구조 변환
last_data = clock_df[-1:]
last_data = last_data.melt()

idcs = last_data[~(last_data['variable'].str.contains('mom')|last_data['variable'].str.contains('trd'))]
trds = last_data[last_data['variable'].str.contains('trd')]
moms = last_data[last_data['variable'].str.contains('mom')]

trds = trds.reset_index(drop=True)
moms = moms.reset_index(drop=True)

idcs['Trend'] = trds['value']
idcs['MoM'] = moms['value']

idcs['size'] = 0.01

In [241]:
fig = px.scatter(idcs, x="MoM", y="Trend", color="variable", size='size', hover_data=idcs[['value']])
fig.show()
## 경기판단: Spring(회복기)
## 10년 장기트랜드 보다 낮지만 금리, 상품지수, 유가, 호주달러가치 측면에서 높은 경기회복이 관찰됨
## 반면 달러지수와 변동성 지수는 장기트랜드 보다 높지만 빠르게 축소되는 것으로 보아 역시 경기회복의 시그널로 볼 수 있음
## 실업 및 고용, 물가지수 관련 데이터는 20년 12월 데이터가 최신